In [1]:
# Useful for being able to dump images into the Notebook
import IPython.display as D

# Policy Recognition with Resnet and Transfer Learning 

In the previous tutorials we outlined our policy classfication challenge and showed how we can approach it using the Custom Vision Cognitive Service. This tutorial introduces deep transfer learning as a means to leverage multiple data sources to overcome data scarcity problem.

# Part 1 Processing Images

We need to be able to represent our images and their classes in a way that a model can understand. To do this we will flatten the images to a vector (28x28 image pixels becomes an array of length 784 data points).

![](https://www.cntk.ai/jup/cntk103a_MNIST_input.png)

We also need a way for the model to know what a three is or what valid vs invalid is. To acomplish this labels are encoded as [1-hot]( https://en.wikipedia.org/wiki/One-hot) encoding (label of 3 with 10 digits becomes `0001000000`, where the first index corresponds to digit `0` and the last one corresponds to digit `9`. When we train a model we use this value as our target. 

![](https://www.cntk.ai/jup/cntk103a_onehot.png)

## Linear Models 

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) (LR) is a fundamental machine learning technique that uses a linear weighted combination of features and generates probability-based predictions of different classes. 
![](https://www.cntk.ai/jup/logistic_neuron2.jpg)

The way this works is that we take our sequence of data pass it through the LR model compare it's numerical output with our onehot label and then use an optimization function to update our linear weights until they converge on the correct reprentation of our data. 

## Non Linear Models (MLP)
As you can imagine the just incrementing one weight value makes it hard to represent our dataset what if we had a way of combining multiple linear models. We can do this with a feed forward network such as the Multi Layer Perceptron Below.

![](https://upload.wikimedia.org/wikipedia/en/5/54/Feed_forward_neural_net.gif)

## What happens when we have a lot of really large images? 

When we have images greater than 500k to 1mb it gets increasingly more computationally expensive to process our image as a sequence additionally it becomes much more challenging to detect complex self referential and hierarchical patterns in our sequence data. 

# Part 2 Traditional Computer Vision 101

What if instead of passing our full image we extracted features and made decisions on those. That is how computer vision worked until recently lets take a deeper look by trying to use edges as a feature for our model.

### Example Masks   

To do this we first we take an image.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Valve_original_%281%29.PNG/300px-Valve_original_%281%29.PNG)

Src [Wikimedia Commons](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Valve_original_%281%29.PNG/300px-Valve_original_%281%29.PNG)

Then we take a [sobel matrix](https://en.wikipedia.org/wiki/Sobel_operator) that is used to find edges.
![](https://www.researchgate.net/publication/275073476/figure/fig6/AS:340842360393754@1458274560103/Sobel-masks-a-horizontal-mask-and-b-vertical-mask.png)

src [researchgate publication #275073476](https://www.researchgate.net/publication/275073476/figure/fig6/AS:340842360393754@1458274560103/Sobel-masks-a-horizontal-mask-and-b-vertical-mask.png)

We apply the sobel matrix to our image in strides like a filter 
![](https://www.cntk.ai/jup/cntk103d_padding_strides.gif)

And when we map the result we get the following edges which we can use as features
![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Valve_sobel_%283%29.PNG/300px-Valve_sobel_%283%29.PNG)
Src [Wikimedia Commons](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Valve_sobel_%283%29.PNG/300px-Valve_sobel_%283%29.PNG)

# Part 3 Intro to the Convolution Neural Network (CNN 101)

Creating these masks by hand is hardwork and brittle what if we could learn them, introducing the CNN.

A CNN is a deep neural network comprised of a bunch of layers in such a way that the output of one layer is fed to the next layer (There are more complex architecture that skip layers with dropout we will take this as a given for now). Usually, CNN's start with alternating between convolution layer and pooling layer (downsample), then end up with fully connected layer for the classification part. 




### Convolve 

A convolution layer is a set of filters. Each filter is defined by a weight (**W**) matrix, and  bias ($b$).

![](https://www.cntk.ai/jup/cntk103d_padding_strides.gif)

### Pool 

Once we apply our mask we use pooling to reduce the dimensionality of the previous layer, which speeds up the network.

![](https://cntk.ai/jup/201/MaxPooling.png)

Here an example of max pooling with a stride of 2:

### Putting it all together

![](https://www.mathworks.com/content/mathworks/www/en/discovery/convolutional-neural-network/jcr:content/mainParsys/image_copy.adapt.full.high.jpg/1508999490138.jpg")

src [mathworks.com](https://www.mathworks.com/content/mathworks/www/en/discovery/convolutional-neural-network/jcr:content/mainParsys/image_copy.adapt.full.high.jpg/1508999490138.jpg)

# Part 4 Intro To Transfer Learning

![](http://www.cntk.ai/jup/cntk301_imagenet.jpg)


### Why Transfer Learning?

Transfer Learning is a useful technique when, for instance, you know you need to classify incoming images into different categories, but you do not have enough data to train a Deep Neural Network (DNN) from scratch. Training DNNs takes a lot of data, all of it labeled, and often you will not have that kind of data on hand. If your problem is similar to one for which a network has already been trained, though, you can use Transfer Learning to modify that network to your problem with a fraction of the labeled images (we are talking tens instead of thousands). 

### What is Transfer Learning?

With Transfer Learning, we use an existing trained model and adapt it to our own problem. We are essentially building upon the features and concepts that were learned during the training of the base model. With a Convolutional DNN (ResNet_18 in this case), we are using the features learned from ImageNet data and _cutting off_ the final classification layer, replacing it with a new dense layer that will predict the class labels of our new domain. 

The input to the old and the new prediction layer is the same, we simply reuse the trained features. Then we train this modified network, either only the new weights of the new prediction layer or all weights of the entire network.

This can be used, for instance, when we have a small set of images that are in a similar domain to an existing trained model. Training a Deep Neural Network from scratch requires tens of thousands of images, but training one that has already learned features in the domain you are adapting it to requires far fewer. 


In our case, this means adapting a network trained on ImageNet images (dogs, cats, birds, etc.) to flowers, or sheep/wolves. However, Transfer Learning has also been successfully used to adapt existing neural models for translation, speech synthesis, and many other domains - it is a convenient way to bootstrap your learning process.



### Pre-Trained Model (ResNet)

For this task, we have chosen ResNet_18 as our trained model and  will it as the base model. This model will be adapted using Transfer Learning for classification of flowers and animals. This model is a [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) built using [Residual Network](https://github.com/KaimingHe/deep-residual-networks) techniques. Convolutional Neural Networks build up layers of convolutions, transforming an input image and distilling it down until they start recognizing composite features, with deeper layers of convolutions recognizing complex patterns are made possible. The author of Keras has a [fantastic post](https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html) where he describes how Convolutional Networks "see the world" which gives a much more detailed explanation.

Residual Deep Learning is a technique that originated in Microsoft Research and involves "passing through" the main signal of the input data, so that the network winds up "learning" on just the residual portions that differ between layers. This has proven, in practice, to allow the training of much deeper networks by avoiding issues that plague gradient descent on larger networks. These cells bypass convolution layers and then come back in later before ReLU (see below), but some have argued that even deeper networks can be built by avoiding even more nonlinearities in the bypass channel. This is an area of hot research right now, and one of the most exciting parts of Transfer Learning is that you get to benefit from all of the improvements by just integrating new trained models.

![](https://adeshpande3.github.io/assets/ResNet.png)

For visualizations of some of the deeper ResNet architectures, see [Kaiming He's GitHub](https://github.com/KaimingHe/deep-residual-networks) where he links off to visualizations of 50, 101, and 152-layer architectures.